In [1]:
import cv2
import pandas as pd
import numpy as np


# set random seed for reproducibility
np.random.seed(42)


# screen resolution
monitor_width = 3440
monitor_height = 1440

In [2]:
def rescale_frame(frame, scale):

    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)

    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def find_scale_factor(frame, monitor_width, monitor_height):
    """
    Rescales the frame if it is larger than the monitor 
    """

    frame_width = frame.shape[1]
    frame_height = frame.shape[0]

    if (frame_width > monitor_width) or (frame_height > monitor_height):
        with_scale = monitor_width / frame_width
        height_scale = monitor_height / frame_height
        return min(with_scale, height_scale)

    return 1
    

In [3]:
df_frames = pd.read_csv('../data/tmp_tracking.csv')

### Pedal position for knee-over-pedal calculations

In [4]:
pedal_coordinates_sampled = []
frames_parallel_pedals = df_frames[df_frames['parallel_pedals'] == True]

# sample a frame from frames_parallel_pedals
sample_frames = frames_parallel_pedals.sample(3)['frame'].values

for i, sample_frame in enumerate(sample_frames):
    img = cv2.imread(f'../data/tmp_frames/frame_{sample_frame}.jpg')

    # does not depend on the monitor resolution
    scale = find_scale_factor(img, monitor_width, monitor_height)
    img_rescaled = rescale_frame(img, scale=scale)


    def capture_event(event, x, y, flags, param):
        captured = False

        if event == cv2.EVENT_MOUSEMOVE:

            # make a copy of the image
            img_rescaled_tmp = img_rescaled.copy()

            # plot a green vertical and horizontal line
            cv2.line(img_rescaled_tmp, (x, 0), (x, img_rescaled.shape[0]), (0, 255, 0), 1)
            cv2.line(img_rescaled_tmp, (0, y), (img_rescaled.shape[1], y), (0, 255, 0), 1)

            cv2.imshow("frame", img_rescaled_tmp)

        if event == cv2.EVENT_LBUTTONDOWN:
            print(f"iteration {i + 1} of 3 - Captured coordinates ({x}, {y}), press any key to continue")
            pedal_coordinates_sampled.append((x, y))
            cv2.destroyWindow("frame")


    cv2.imshow('frame', img_rescaled)
    cv2.setMouseCallback('frame', capture_event)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

pedal_coordinates = np.array(pedal_coordinates_sampled).mean(axis=0).astype(int)
print(f"Pedal coordinates obtained: {pedal_coordinates}")

iteration 0 of 3 - Captured coordinates (163, 467), press any key to continue
iteration 1 of 3 - Captured coordinates (161, 461), press any key to continue
iteration 2 of 3 - Captured coordinates (163, 462), press any key to continue
